In [1]:
%pip install --quiet langchain
%pip install --quiet chroma-hnswlib
%pip install --quiet lark
%pip install --quiet HuggingFace
%pip install --quiet panel
%pip install --quiet -U langchain-community
%pip install --quiet python-dotenv
%pip install --quiet pypdf
%pip install --quiet sentence_transformers
%pip install --quiet chromadb



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[noti

In [2]:
import os
import numpy as np
import sys
import chromadb
import dotenv
sys.path.append('../..')

import panel as pn  # GUI
pn.extension()

from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.embeddings import HuggingFaceBgeEmbeddings


In [3]:
persist_directory = './database/chroma'

In [4]:
!rm -rf ./database/chroma # remove old database files if any

In [5]:
loader = PyPDFLoader("Paper.pdf")
pages = loader.load()

In [6]:
full_text = " ".join([page.page_content for page in pages])

# Initialize the RecursiveCharacterTextSplitter
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""]
)

# Split the text into chunks
chunks = r_splitter.split_text(full_text)


<>:7: SyntaxWarning: invalid escape sequence '\.'
<>:7: SyntaxWarning: invalid escape sequence '\.'
/var/folders/sd/bjm5gr593mb__5xtlb54wqbr0000gn/T/ipykernel_20354/4004144305.py:7: SyntaxWarning: invalid escape sequence '\.'
  separators=["\n\n", "\n", "(?<=\. )", " ", ""]


In [7]:
chunks[1] #just to check if it's working

'in both deep learning architectural and evaluation approaches, and dataset building for easy\nmethods comparison.\nKeywords Image forgery detection ·Image forensics ·Deep learning ·Copy-move ·\nSplicing ·DeepFake ·Survey\n/envelopebackMarcello Zanardelli\nm.zanardelli005@unibs.it\nFabrizio Guerrini\nfabrizio.guerrini@unibs.it\nRiccardo Leonardi\nriccardo.leonardi@unibs.it\nNicola Adami\nnicola.adami@unibs.it\n1Department of Information Engineering, CNIT – University of Brescia,\nVia Branze 38, 25134 Brescia, ItalyPublished online: 3 October 2022Multimedia Tools and Applications (2023) 82:17521–17566 1 Introduction\nThe worldwide spread of smart devices, which integrate increasing quality cameras and\nimage processing tools and “apps”, the ubiquity of desktop computers, and the fact that allthese devices are almost permanently connected with each other and to remotely locateddata servers through the Internet, have given ordinary people the possibility to collect, store,and process an e

In [8]:
model_name = "BAAI/bge-large-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    query_instruction="为这个句子生成表示以用于检索相关文章："
)
model.query_instruction = "为这个句子生成表示以用于检索相关文章："


/opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [9]:
# Assuming 'chunks' is a list of strings
documents = [Document(page_content=chunk) for chunk in chunks]

vectordb = Chroma.from_documents(
    documents=documents,  # Pass the list of Document objects
    embedding=model,
    persist_directory=persist_directory,
    client=chromadb.Client()
)

In [10]:
print(vectordb._collection.count())

103


In [116]:
query = "who are the authors of this survey?"


In [69]:
%pip install --quiet pandas 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [70]:
import pandas as pd

In [117]:
docs = vectordb.max_marginal_relevance_search(query)

# print results
for doc in docs:
    df = pd.DataFrame(doc)

    print(doc.page_content)


are particularly representative of those that can be categorized into at least one of the distinctgroups that we have outlined above. A further principle that we have used for this selectionis performance driven, with the added objective of being able to do a meaningful compar-ison (when possible), given in Section 4. These papers are described in some detail in this17532 Multimedia Tools and Applications (2023) 82:17521–17566 Section, with the further objective of identifying if any trend in the DL overall architecture
choice is emerging.
In particular, we have used the criteria A) and B) above to sort the presentation order of
the papers. Methods [ 1,4,25,78], and [105] are copy-move-only specific, and are presented
first in Section 3.2. Then, methods [ 22,68,85,86,97,105, 107], and [18], that are for both
splicing and copy-move detection, are discussed next in Section 3.3.
Besides this first separation through criterion A), we sort the techniques in each subset
using criterion B), n

In [102]:
%pip install --quiet langchain-google-genai==0.0.6


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Cannot uninstall packaging 24.0, RECORD file not found. Hint: The package was installed by brew.

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [103]:
from langchain import PromptTemplate
from langchain import hub
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import Chroma
     

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [105]:
%pip install --quiet langchain_google_genai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [107]:
from langchain_google_genai import ChatGoogleGenerativeAI

# To configure model parameters use the `generation_config` parameter.
# eg. generation_config = {"temperature": 0.7, "topP": 0.8, "topK": 40}
# If you only want to set a custom temperature for the model use the
# "temperature" parameter directly.

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", google_api_key=os.getenv('API_KEY'))

In [108]:
# Prompt template to query Gemini
llm_prompt_template = """You are an assistant for question-answering tasks.
Use the following context to answer the question.
If you don't know the answer, just say that you don't know.
Use five sentences maximum and keep the answer concise.\n
Question: {question} \nContext: {context} \nAnswer:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

input_variables=['context', 'question'] template="You are an assistant for question-answering tasks.\nUse the following context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse five sentences maximum and keep the answer concise.\n\nQuestion: {question} \nContext: {context} \nAnswer:"


In [113]:

# Get the Retriever interface for the store to use later.
# When an unstructured query is given to a retriever it will return documents.
# Read more about retrievers in the following link.
# https://python.langchain.com/docs/modules/data_connection/retrievers/
#
# Since only 1 document is stored in the Chroma vector store, search_kwargs `k`
# is set to 1 to decrease the `k` value of chroma's similarity search from 4 to
# 1. If you don't pass this value, you will get a warning.
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

# Check if the retriever is working by trying to fetch the relevant docs related
# to the word 'MMLU' (Massive Multitask Language Understanding). If the length is greater than zero, it means that
# the retriever is functioning well.
print(len(retriever.get_relevant_documents("MMLU")))

1


/opt/homebrew/Cellar/jupyterlab/4.2.1/libexec/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [114]:
# Combine data from documents to readable string format.
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Create stuff documents chain using LCEL.
#
# This is called a chain because you are chaining together different elements
# with the LLM. In the following example, to create the stuff chain, you will
# combine the relevant context from the website data matching the question, the
# LLM model, and the output parser together like a chain using LCEL.
#
# The chain implements the following pipeline:
# 1. Extract the website data relevant to the question from the Chroma
#    vector store and save it to the variable `context`.
# 2. `RunnablePassthrough` option to provide `question` when invoking
#    the chain.
# 3. The `context` and `question` are then passed to the prompt where they
#    are populated in the respective variables.
# 4. This prompt is then passed to the LLM (`gemini-pro`).
# 5. Output from the LLM is passed through an output parser
#    to structure the model's response.
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)
     

In [119]:
rag_chain.invoke("What is the summary of the document?")

'The document is a collection of references related to multimedia tools, applications, and techniques. It includes references to specific software like Adobe Photoshop, research papers on topics such as copy-move forgery detection, and technical reports on superpixel algorithms. The references cover various aspects of digital image processing and security, including feature extraction, steganography, and forensic analysis. \n'

In [22]:
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.getenv("YOUR_ANTHROPIC_API_KEY")
)

message = client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": "Hello, Claude"}
    ]
)
print(message.content)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

In [25]:
%pip install google-generativeai

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
"""
Install the Google AI Python SDK

$ pip install google-generativeai

See the getting started guide for more information:
https://ai.google.dev/gemini-api/docs/get-started/python
"""

import os

import google.generativeai as genai

genai.configure(api_key=os.environ["API_KEY"])

# Create the model
# See https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

chat_session = model.start_chat(
  history=[
  ]
)

response = chat_session.send_message(query, docs)

print(response.text)

TypeError: ChatSession.send_message() takes 2 positional arguments but 3 were given

In [30]:
%pip install --quiet textwrap

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement textwrap (from versions: none)
ERROR: No matching distribution found for textwrap

[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [39]:
%pip install --quiet pandas

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [72]:
#df = pd.DataFrame(docs)
#df.columns = ['Text']
df
    

,0,1
0,id,None
1,metadata,{}
2,page_content,CASIAv1.0(CASIA1) [24] It contains 1725 color ...
3,type,Document


In [98]:
import textwrap
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = textwrap.dedent("""You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [99]:
prompt = make_prompt(query, df.loc[0, "page_content"])
print(prompt)

KeyError: 'page_content'